## 파인콘 벡터 DB 저장
**A**

In [1]:
%whos

Interactive namespace is empty.


In [2]:
from dotenv import load_dotenv
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter 
from langchain_community.document_loaders import Docx2txtLoader
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone
import os

## 1. 문서 내용 읽고 분할
loader = Docx2txtLoader('law_2.docx') #인스턴스 생성 #문서 확인

text_splitter = RecursiveCharacterTextSplitter(  #분할을 하기위한 설정
    chunk_size=1500,
    chunk_overlap=200,
) #나중에 여러번의 질문을 통해 정확도를 확인
document_list = loader.load_and_split(text_splitter=text_splitter) #파일을 갖고 파라미터에 설정된 읽고 분할

## 2. 임베딩 -> 벡터 데이터베이스에 저장 
## 2.1. 환경변수 읽어오기
load_dotenv() #잘 읽어왔다는 True

## 2.2. 임베딩 모델 지정
embedding = OpenAIEmbeddings(model='text-embedding-3-large') #성능 좋은 유료  # 위에서 OPENAI_API_KEY 읽었기 때문에 


c:\Users\71\miniconda3\envs\project\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

# # ## 2.3. 벡터 데이터베이스에 저장 ########한번 실행했기 때문에
##[방법 1] in memory
# database = Chroma.from_documents(
#     documents=document_list,
#     embedding=embedding,
#     persist_directory='./chroma',
#     collection_name='chroma-law',
# )

## 로컬에 저장된 임베딩 데이터 가져오기

# database =Chroma(
#     collection_name='chroma-law',
#     persist_directory='./chroma',
#     embedding_function=embedding,
# )

##[방법 2] pinecone: server
api_key = os.getenv('PINECONE_API_KEY')
pc = Pinecone(api_key=api_key)
pc


In [4]:

database = PineconeVectorStore.from_documents(
    documents=document_list,
    embedding=embedding,
    index_name='law-2',
)

database

In [9]:



## 3. 질문이 있으면, 벡터 데이터베이스(vector_store)에서 유사도 검색

## 3.1. 사용자 질문
# query = '오늘 서울 날씨는?'
# query = '전세사기 피해자는 누구인가요? '
# query = '전세사기 피해주택 임대인의 국세 계산은 어떻게 하나요?'
query = '전세사기 피해 주택의 가격은?'

## 3.2. 벡터 데이터베이스에서 유사도 검색 (점수 포함)
retrieved_docs = database.similarity_search(query=query, k=2)

## 문서 객체에서 문자열만 추출



## 3.3 문서 객체 -> 하나의 문자열
## 리스트 for 문
context = '🤷‍♀️🤷‍♀️\n\n'.join([doc.page_content for doc in retrieved_docs])
context



'전세사기피해자 지원 및 주거안정에 관한 특별법 시행령 ( 약칭: 전세사기피해자법 시행령 )\n\n[시행 2024. 11. 11.] [대통령령 제34987호, 2024. 11. 8., 일부개정]\n\n\n\n국토교통부(피해지원총괄과) 044-201-5233, 5234\n\n\n\n제1조(목적) 이 영은 「전세사기피해자 지원 및 주거안정에 관한 특별법」에서 위임된 사항과 그 시행에 필요한 사항을 규정함을 목적으로 한다.\n\n제2조(전세사기피해지원단) ① 「전세사기피해자 지원 및 주거안정에 관한 특별법」(이하 “법”이라 한다) 제10조제1항에 따른 전세사기피해지원단(이하 “지원단”이라 한다)은 단장 1명과 단원으로 구성한다.\n\n  ② 지원단의 단장은 국토교통부의 고위공무원단에 속하는 일반직공무원 중에서 국토교통부장관이 지명하는 사람이 된다.\n\n  ③ 지원단의 단장은 국토교통부장관의 명을 받아 지원단의 업무를 총괄하고, 지원단의 단원을 지휘ㆍ감독한다.\n\n  ④ 제1항부터 제3항까지에서 규정한 사항 외에 지원단의 구성 및 운영 등에 필요한 사항은 국토교통부장관이 정한다.\n\n제3조(국세의 안분 방법 및 신청 등) ① 법 제23조제1항 각 호 외의 부분 전단에 따라 안분하여 징수하는 전세사기피해주택 임대인의 국세는 다음 각 호의 구분에 따라 산정한다.\n\n  1. 상속세, 증여세 및 종합부동산세의 경우: 고지 또는 신고 건별로 각각 가목에 따라 계산한 금액과 나목의 금액 중 큰 금액\n\n    가. 다음의 계산식에 따라 계산한 금액\n\n  \n\nA × (B / C)\n\n- **A**: 전세사기피해주택의 임대인이 체납한 고지 또는 신고 건별 국세 금액  \n\n- **B**: 전세사기피해주택의 가격  \n\n- **C**: 전세사기피해주택의 임대인이 보유한 모든 주택의 가격 합계액  \n\n\n\n\n\n    나. 전세사기피해주택의 임대인이 체납한 고지 또는 신고 건별 국세 금액 중 「국세기본법」 제35조제3항에 따른 해당 재산에 대하여 부과된 

In [ ]:
retrieved_docs

In [11]:

## 4.0 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달 

## 4.1. 프롬프트 작성
prompt = '''
[identity]
- 당신은 전세사기 피해 법률 전문가입니다.
- [context]를 참고하여 사용자의 질문에 답변해주세요.
- 전세사기 피해 법률 이외의 질문에는 '답변을 할 수 없습니다.'로 답하세요.

[context]
{retrieved_docs}

Question: {query}
'''

## 4.2. 프롬프트 변수에 값 설정
formatted_prompt = prompt.format(
    retrieved_docs=context,
    query=query,
)

## 4.3. LLM 모델 생성(chatOpenAI 인스턴스 생성)  
llm = ChatOpenAI(model='gpt-4o')

## 4.4. LLM 모델에 질문과 검색된 문서를 보냄  
ai_message = llm.invoke(formatted_prompt)
ai_message.content

'전세사기 피해 주택의 가격은 "지방세법" 제114조에 따른 과세기준일에 해당 주택의 소유자가 소유하고 있었던 주택의 "지방세법" 제4조제1항 및 제2항에 따른 시가표준액을 기준으로 산정됩니다. 이 시가표준액은 "지방세법" 제110조제1항에 따라 해당 주택에 대한 재산세의 과세표준 산정에 적용된 금액을 의미합니다.'

In [ ]:
retrieved_docs